In [1]:
import pandas as pd

Unemployment Data by county (https://www.bls.gov/lau/#tables)

In [2]:
list_ = []
for x in range(2010,2019):
    df = pd.read_fwf(f'https://www.bls.gov/lau/laucnty{str(x)[-2:]}.txt', skiprows = 5)
    list_.append(df)
all_data = pd.concat(list_)
all_data = all_data.dropna()
all_data = all_data.rename(columns = {
    'Unnamed: 3' : 'county_name',
    'Unnamed: 4' : 'year',
    'Unnamed: 8' : 'unemployment_rate'
})

In [3]:
all_data.year = all_data.year.astype(int)
unemployment_data = all_data[['year','county_name','unemployment_rate']]
# unemployment_data[['county','state']] = unemployment_data.county_name.str.split(',', expand = True)
# unemployment_data = unemployment_data[['year', 'county', 'state', 'unemployment_rate']]

In [5]:
zip_county_key = pd.read_csv('zip_county_key.csv')
zip_county_key = zip_county_key.rename(columns = {
    'COUNTYNAME' : 'county',
    'STATE' : 'state'
})
zip_county_key['county_name'] = zip_county_key['county'] + ", " + zip_county_key['state']

In [6]:
unemployment_data = pd.merge(unemployment_data,
             zip_county_key,
             how = 'left',
             on = ['county_name'])
unemployment_data = unemployment_data[['year', 'ZIP', 'unemployment_rate']]

In [7]:
unemployment_data = unemployment_data.rename(columns = {'year': 'Year', 'ZIP_x' : 'ZIP'})

In [9]:
unemployment_data.to_csv('unemployment_data.csv')

Income Data by County (https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-zip-code-data-soi)

In [10]:
# df = pd.read_csv('https://www.irs.gov/pub/irs-soi/16zpallagi.csv')[['zipcode', 'agi_stub', 'N1']]

In [8]:
list_ = []
for x in range(2014,2017):
    df = pd.read_csv(f'https://www.irs.gov/pub/irs-soi/{str(x)[-2:]}zpallagi.csv')[['zipcode', 'agi_stub', 'N1']]
    list_.append(df)
    df['year'] = x
income_master = pd.concat(list_)

In [2]:
income_master = pd.read_csv('income_placeholder.csv')

In [3]:

income_master

,Unnamed: 0,zipcode,agi_stub,N1,year
0,0,0,1,850050.0,2014
1,1,0,2,491370.0,2014
2,2,0,3,259540.0,2014
3,3,0,4,164840.0,2014
4,4,0,5,203650.0,2014
5,5,0,6,51270.0,2014
6,6,35004,1,1510.0,2014
7,7,35004,2,1280.0,2014
8,8,35004,3,920.0,2014
9,9,35004,4,630.0,2014


In [4]:
income = []


for index, row in income_master.iterrows():
    if row['agi_stub'] == 1:
        income.append(12500)
    elif row['agi_stub'] == 2:
        income.append(37500)
    elif row['agi_stub'] == 3:
        income.append(62500)
    elif row['agi_stub'] == 4:
        income.append(87500)
    elif row['agi_stub'] == 5:
        income.append(150000)
    elif row['agi_stub'] == 6:
        income.append(200000)
        
income_master['income'] = income

In [22]:
def wavg(val_col_name, wt_col_name):
    def inner(group):
        return (group[val_col_name] * group[wt_col_name]).sum() / group[wt_col_name].sum()
    inner.__name__ = 'wtd_avg'
    return inner

income_master_test= pd.DataFrame(income_master.groupby(['zipcode', 'year']).apply(wavg('income', 'N1')))

In [23]:
# income_master_test = income_master_test['WAVG']

In [25]:
income_master_test.to_csv('test_income.csv', index=True)